## Imports

In [1]:
print('')

In [2]:
import os
import math
import obspy
import pickle
import datetime
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.pyplot import figure
from scipy.optimize import curve_fit
from datetime import timedelta
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

from earthquake import Earthquake
import util
import pandas as pd

## Get set up
Set path to data, and read eq_list (all folders in root folder)

In [3]:
root = '/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3/'

eq_list = os.listdir(root)

Open catalog of events

In [4]:
client = Client("IRIS")
# cat = client.get_events(starttime=UTCDateTime("2019-01-01"), endtime=UTCDateTime("2020-01-01"), minmagnitude=5, includearrivals=True)
cat = obspy.read_events('/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3_catalog.xml')

Not all events had suitable data, look through all events and make a list of ones which have data (eq_with_data) and were successfully picked

In [5]:
eq_with_data = []
cat_with_data = cat.copy()
cat_with_data.clear()
for event in cat:
    eq_name = util.catEventToFileName(event)
    if os.path.isdir(root+eq_name) and os.path.isdir(root+eq_name+'/station_xml_files') and os.path.exists(root+eq_name+'/picks.pkl'):
        eq_with_data.append(eq_name)
        cat_with_data.extend([event])

## the action!!


find location of tpmax in time

In [6]:
client = Client("IRIS")
# cat = client.get_events(starttime=UTCDateTime("2019-01-01"), endtime=UTCDateTime("2020-01-01"), minmagnitude=5, includearrivals=True)
cat = obspy.read_events('/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3_catalog.xml')

In [7]:
eq_with_data = []
cat_with_data = cat.copy()
cat_with_data.clear()
for event in cat:
    eq_name = util.catEventToFileName(event)
    if os.path.isdir(root+eq_name) and os.path.isdir(root+eq_name+'/station_xml_files') and os.path.exists(root+eq_name+'/picks.pkl'):
        eq_with_data.append(eq_name)
        cat_with_data.extend([event])
print(len(eq_with_data))

6075


In [8]:
%matplotlib qt 
WINDOW_LENGTH = 4
diff = []
for eq_no in range(0, 10):#len(eq_with_data)):
    print(eq_no)
    with open('/home/earthquakes1/homes/Rebecca/phd/data/2019_global_m3/'+eq_with_data[eq_no]+'/eq_object.pkl', 'rb') as picklefile:
        eq = pickle.load(picklefile)
        keys = list(eq.data_stats['picks'].keys())
        for i in range(len(eq.calculated_params['tau_p_max'])):
            name = eq.calculation_info["tau_p_stations"][i]
            if name in eq.data_stats['picks'].keys():
                pick = (eq.data_stats['picks'][name]-eq.data[i*3+2].stats.starttime)*eq.data[i*3+2].stats.sampling_rate
                #plt.plot(eq.calculated_params['tau_p'][i])
                #plt.axhline(eq.calculated_params['tau_p_max'][i], 0, 1000)
                #plt.show()
                pick_seconds = pick / (eq.data[i*3].stats['sampling_rate'])
                a = np.where(eq.calculated_params['tau_p'][i][int(pick):]==eq.calculated_params['tau_p_max'][i])
                if len(a[0])>0:
                    max_loc = a[0][0]* 100/(eq.data[i*3+2].stats['sampling_rate'])+pick * 100/(eq.data[i*3+2].stats['sampling_rate'])
                    #print(eq.data[i*3])
                    if abs(max_loc-pick* 100/(eq.data[i*3+2].stats['sampling_rate'])) < 4 * eq.data[i*3].stats['sampling_rate']:
                        #eq.data[i*3+2].plot()
                        #plt.plot(eq.calculated_params['tau_p'][i][int(pick):])
                        #plt.axhline(eq.calculated_params['tau_p_max'][i], 0, 1000)
                        #plt.show()
                        #print(i)
                        #print(eq.calculated_params['tau_p_max'][i])
                        #print(max(eq.calculated_params['tau_p'][i][int(pick)+50:int(pick)+400]))
                        #print(eq.data[i*3+2].stats.sampling_rate)
                        #print(pick)
                        #print(max_loc)
                        #print(max_loc-pick* 100/(eq.data[i*3].stats['sampling_rate']))
                        diff.append((max_loc-pick*100/(eq.data[i*3].stats['sampling_rate']))/100)
    #list_tpmax.append(eq.calculated_params['tau_p_max'])

    #except Exception:
    #    print('in except')
    #    continue

0


AttributeError: 'Earthquake' object has no attribute 'data'

In [ ]:
plt.hist(diff, np.arange(0, 4, 0.1))
plt.ylabel('time from pick')
plt.xlabel('n')
plt.title('timing of tpmax')

In [104]:
eq.calculated_params['tau_p'][i][int(pick):]

array([ 0.36418575,  0.36451736,  0.36515183, ...,  4.57211618,
        4.55904055,  4.54593945])

In [105]:
eq.calculated_params['tau_p_max'][i]

0.53657723300601767

In [11]:
eq.__dict__


{'event_stats': {'name': '20190131_194949.a',
  'eq_lat': -20.4865,
  'eq_long': -69.0165,
  'eq_depth': 102120.0,
  'eq_mag': 4.2,
  'eq_mag_type': 'mb'},
 'event': Event:	2019-01-31T19:49:49.660000Z | -20.486,  -69.016 | 4.2  mb

	            resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?eventid=11001567")
	             event_type: 'earthquake'
	    preferred_origin_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=36907036")
	 preferred_magnitude_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=192541937")
	                   ---------
	     event_descriptions: 1 Elements
	                origins: 1 Elements
	             magnitudes: 1 Elements,
 'data_stats': {'picks': {'CX.PB08.': 2019-01-31T19:50:06.058394Z,
   'CX.PB01.': 2019-01-31T19:50:08.498393Z,
   'CX.HMBCX.': 2019-01-31T19:50:09.489999Z},
  'sensor_types': ['VEL',
   'VEL',
   'VEL',
   'VEL',
   'VEL',
   'VEL',
   'VEL',
   'VEL'